In [ ]:
#在本节中，我们将讨论使用自注意力进行序列编码，包括使用序列的顺序作为补充信息。
!pip install d2l
import math
import torch
from torch import nn
from d2l import torch as d2l

In [ ]:
#10.6.1. 自注意力
#根据 (10.2.4) 中定义的注意力池化函数  f 。
#下面的代码片段是基于多头注意力对一个张量完成自注意力的计算，张量的形状为 (批量大小, 时间步的数目或词元序列的长度,  d ) 。
#输出与输入的张量形状相同。
num_hiddens, num_heads = 100, 5
attention = d2l.MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                                   num_hiddens, num_heads, 0.5)
attention.eval()

In [ ]:
batch_size, num_queries, valid_lens = 2, 4, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
attention(X, X, X, valid_lens).shape

In [ ]:
#10.6.3. 位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, num_hiddens, dropout, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        # 创建一个足够长的 `P`
        self.P = torch.zeros((1, max_len, num_hiddens))
        X = torch.arange(max_len, dtype=torch.float32).reshape(
            -1, 1) / torch.pow(10000, torch.arange(
            0, num_hiddens, 2, dtype=torch.float32) / num_hiddens)
        self.P[:, :, 0::2] = torch.sin(X)
        self.P[:, :, 1::2] = torch.cos(X)

    def forward(self, X):
        X = X + self.P[:, :X.shape[1], :].to(X.device)
        return self.dropout(X)

In [ ]:
#在位置嵌入矩阵  P  中，行代表词元在序列中的位置，列代表位置编码的不同维度。
#在下面的例子中，我们可以看到位置嵌入矩阵的 第 6 列 和 第 7 列的频率高于 第 8  列和 第 9  列。
#第 6 列 和 第 7 列之间的偏移量（第 8  列和 第 9  列相同）是由于正弦函数和余弦函数的交替。
encoding_dim, num_steps = 32, 60
pos_encoding = PositionalEncoding(encoding_dim, 0)
pos_encoding.eval()
X = pos_encoding(torch.zeros((1, num_steps, encoding_dim)))
P = pos_encoding.P[:, :X.shape[1], :]
d2l.plot(torch.arange(num_steps), P[0, :, 6:10].T, xlabel='Row (position)',
         figsize=(6, 2.5), legend=["Col %d" % d for d in torch.arange(6, 10)])

In [ ]:
#10.6.3.1. 绝对位置信息
#为了明白沿着编码维度单调降低的频率与绝对位置信息的关系，让我们打印出  0,1,…,7  
#的二进制表示形式。正如我们所看到的，每个数字、每两个数字和每四个数字上的比特值在第一个最低位、第二个最低位和第三个最低位上分别交替。
for i in range(8):
    print(f'{i} in binary is {i:>03b}')

In [ ]:
#在二进制表示中，较高比特位的交替频率低于较低比特位，与下面的热图所示相似，只是位置编码通过使用三角函数在编码维度上降低频率。
#由于输出是浮点数，因此此类连续表示比二进制表示法更节省空间。
P = P[0, :, :].unsqueeze(0).unsqueeze(0)
d2l.show_heatmaps(P, xlabel='Column (encoding dimension)',
                  ylabel='Row (position)', figsize=(3.5, 4), cmap='Blues')